<a href="https://colab.research.google.com/github/pbeles/lab-sql-query-from-table-names/blob/main/lab-sql-query-from-table-names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [2]:
from openai import OpenAI
from google.colab import userdata

OPENAI_API_KEY  = userdata.get('chat_gpt')

In [3]:
#Function to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [4]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['employees', 'salary', 'studies'],
        'definition': ['Employee information, name...',
                      'Salary details for each year',
                      'Educational studies, name of the institution, ...']}

df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees                      Employee information, name...
1     salary                       Salary details for each year
2    studies  Educational studies, name of the institution, ...


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [6]:
print(text_tables)

employees: Employee information, name...
salary: Salary details for each year
studies: Educational studies, name of the institution, ...


In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [8]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="List employees and their salaries")


In [9]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "salary"]
}
```


In [10]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Find employees with their education level and current salary")

In [11]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "salary", "studies"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [12]:
#Version 1 - Basic Table List:
pqt5 = prompt_question_tables.format(tables=text_tables,
                                   question="Compare employee salary progression year-over-year and their highest education level")
#Version 2 - Educational Focus:
pqt6 = prompt_question_tables.format(tables=text_tables,
                                   question="Find employees with PhD degrees and their current salaries")
#Version 3 - Complex Join:
pqt7 = prompt_question_tables.format(tables=text_tables,
                                   question="Show employees with Master's degrees earning above average salary")
print(return_OAI(pqt5))
print(return_OAI(pqt6))
print(return_OAI(pqt7))


```json
{
    "tables": ["employees", "salary", "studies"]
}
```
```json
{
    "tables": ["employees", "studies", "salary"]
}
```
```json
{
    "tables": ["employees", "salary", "studies"]
}
```


🤔 What I Learned:

Data Structure Matters


Simple prompts = cleaner JSON outputs
Complex queries needed all three tables
DataFrame setup impacts response quality


Missing specific queries led to empty responses
Table relationships weren't always clear
Temperature=0 was crucial for consistent JSON

##takeaways
JSON format made parsing super easy
Educational queries showed best table relationships
Salary queries needed explicit year mentions

# Always set temperature=0 for consistent format
# Keep table definitions clear
# Test with simple queries first
Next time I'd:

Add more example queries
Include data types
Test edge cases more

Learning how prompts shape AI responses.  